# Advanced Retrieval with LangChain

In the following notebook, we'll explore various methods of advanced retrieval using LangChain!

We'll touch on:

- Naive Retrieval
- Best-Matching 25 (BM25)
- Multi-Query Retrieval
- Parent-Document Retrieval
- Contextual Compression (a.k.a. Rerank)
- Ensemble Retrieval
- Semantic chunking

We'll also discuss how these methods impact performance on our set of documents with a simple RAG chain.

There will be two breakout rooms:

- 🤝 Breakout Room Part #1
  - Task 1: Getting Dependencies!
  - Task 2: Data Collection and Preparation
  - Task 3: Setting Up QDrant!
  - Task 4-10: Retrieval Strategies
- 🤝 Breakout Room Part #2
  - Activity: Evaluate with Ragas

# 🤝 Breakout Room Part #1

## Task 1: Getting Dependencies!

We're going to need a few specific LangChain community packages, like OpenAI (for our [LLM](https://platform.openai.com/docs/models) and [Embedding Model](https://platform.openai.com/docs/guides/embeddings)) and Cohere (for our [Reranker](https://cohere.com/rerank)).

> You do not need to run the following cells if you are running this notebook locally. 

In [ ]:
#!pip install -qU langchain langchain-openai langchain-cohere rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.1/233.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.1/378.1 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.2 MB/s eta 0:00:00


We're also going to be leveraging [Qdrant's](https://qdrant.tech/documentation/frameworks/langchain/) (pronounced "Quadrant") VectorDB in "memory" mode (so we can leverage it locally in our colab environment).

In [ ]:
#!pip install -qU qdrant-client

We'll also provide our OpenAI key, as well as our Cohere API key.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key:")

In [2]:
os.environ["COHERE_API_KEY"] = getpass.getpass("Cohere API Key:")

## Task 2: Data Collection and Preparation

We'll be using some reviews from the 4 movies in the John Wick franchise today to explore the different retrieval strategies.

These were obtained from IMDB, and are available in the [AIM Data Repository](https://github.com/AI-Maker-Space/DataRepository).

### Data Collection

We can simply `wget` these from GitHub.

You could use any review data you wanted in this step - just be careful to make sure your metadata is aligned with your choice.

In [3]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv -O john_wick_1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv -O john_wick_2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw3.csv -O john_wick_3.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw4.csv -O john_wick_4.csv

--2025-05-16 10:10:59--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19628 (19K) [text/plain]
Saving to: ‘john_wick_1.csv’

john_wick_1.csv     100%[===================>]  19.17K  --.-KB/s    in 0.004s  

2025-05-16 10:10:59 (4.60 MB/s) - ‘john_wick_1.csv’ saved [19628/19628]

--2025-05-16 10:11:00--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/jw2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14747 (14K) [text/plain]
Sa

### Data Preparation

We want to make sure all our documents have the relevant metadata for the various retrieval strategies we're going to be applying today.

- Self-Query: Wants as much metadata as we can provide
- Time-weighted: Wants temporal data

> NOTE: While we're creating a temporal relationship based on when these movies came out for illustrative purposes, it needs to be clear that the "time-weighting" in the Time-weighted Retriever is based on when the document was *accessed* last - not when it was created.

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

Let's look at an example document to see if everything worked as expected!

In [5]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 5, 13, 10, 11, 7, 491393)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

In [6]:
len(documents)

100

## Task 3: Setting up QDrant!

Now that we have our documents, let's create a QDrant VectorStore with the collection name "JohnWick".

We'll leverage OpenAI's [`text-embedding-3-small`](https://openai.com/blog/new-embedding-models-and-api-updates) because it's a very powerful (and low-cost) embedding model.

> NOTE: We'll be creating additional vectorstores where necessary, but this pattern is still extremely useful.

In [7]:
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

vectorstore = Qdrant.from_documents(
    documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWick"
)

## Task 4: Naive RAG Chain

Since we're focusing on the "R" in RAG today - we'll create our Retriever first.

### R - Retrieval

This naive retriever will simply look at each review as a document, and use cosine-similarity to fetch the 10 most relevant documents.

> NOTE: We're choosing `10` as our `k` here to provide enough documents for our reranking process later

In [8]:
naive_retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

### A - Augmented

We're going to go with a standard prompt for our simple RAG chain today! Nothing fancy here, we want this to mostly be about the Retrieval process.

In [9]:
from langchain_core.prompts import ChatPromptTemplate

RAG_TEMPLATE = """\
You are a helpful and kind assistant. Use the context provided below to answer the question.

If you do not know the answer, or are unsure, say you don't know.

Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

### G - Generation

We're going to leverage `gpt-4.1-nano` as our LLM today, as - again - we want this to largely be about the Retrieval process.

In [10]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4.1-nano")

### LCEL RAG Chain

We're going to use LCEL to construct our chain.

> NOTE: This chain will be exactly the same across the various examples with the exception of our Retriever!

In [11]:
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

naive_retrieval_chain = (
    # INVOKE CHAIN WITH: {"question" : "<<SOME USER QUESTION>>"}
    # "question" : populated by getting the value of the "question" key
    # "context"  : populated by getting the value of the "question" key and chaining it into the base_retriever
    {"context": itemgetter("question") | naive_retriever, "question": itemgetter("question")}
    # "context"  : is assigned to a RunnablePassthrough object (will not be called or considered in the next step)
    #              by getting the value of the "context" key from the previous step
    | RunnablePassthrough.assign(context=itemgetter("context"))
    # "response" : the "context" and "question" values are used to format our prompt object and then piped
    #              into the LLM and stored in a key called "response"
    # "context"  : populated by getting the value of the "context" key from the previous step
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's see how this simple chain does on a few different prompts.

> NOTE: You might think that we've cherry picked prompts that showcase the individual skill of each of the retrieval strategies - you'd be correct!

In [12]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked John Wick. The majority of reviews rate the film highly, praising its action sequences, style, and Keanu Reeves' performance. Many reviewers expressed that it is an exciting and well-crafted action movie that has been well received by audiences. However, there are some mixed opinions with a few ratings that are more moderate, but overall, the consensus appears to be positive."

In [13]:
naive_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there are reviews with a rating of 10. The URLs to those reviews are:\n\n1. [Review URL: /review/rw4854296/?ref_=tt_urv](https://example.com/review/rw4854296/?ref_=tt_urv)\n2. [Review URL: /review/rw4860412/?ref_=tt_urv](https://example.com/review/rw4860412/?ref_=tt_urv)\n\nPlease note that the URLs provided are based on the review URLs given in the context. You can access the reviews through these links.'

In [14]:
naive_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In the John Wick series, the story revolves around John Wick, a retired hitman who seeks vengeance after a series of personal losses and attacks. The first film depicts how Wick comes out of retirement after gangsters kill his dog and steal his car, which forces him to confront his dark past and unleash a deadly assault on those who wronged him. Throughout the series, Wick's quest for revenge and justice leads to intense action sequences, complex criminal underworlds, and unavoidable consequences, depicting him as a legendary and unstoppable assassin."

Overall, this is not bad! Let's see if we can make it better!

## Task 5: Best-Matching 25 (BM25) Retriever

Taking a step back in time - [BM25](https://www.nowpublishers.com/article/Details/INR-019) is based on [Bag-Of-Words](https://en.wikipedia.org/wiki/Bag-of-words_model) which is a sparse representation of text.

In essence, it's a way to compare how similar two pieces of text are based on the words they both contain.

This retriever is very straightforward to set-up! Let's see it happen down below!


In [15]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(documents)

We'll construct the same chain - only changing the retriever.

In [16]:
bm25_retrieval_chain = (
    {"context": itemgetter("question") | bm25_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at the responses!

In [17]:
bm25_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked the John Wick movies, especially the first one, which received high ratings and positive comments highlighting its stylish action and engaging world. However, opinions on later installments vary, with some viewer reviews expressing dissatisfaction, especially with the third and fourth movies. Overall, there is a trend of appreciation for the franchise's action and style, but not universally."

In [18]:
bm25_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Based on the provided reviews, none of the reviews have a rating of 10. The highest ratings mentioned are 9, with some reviews at ratings of 1 and 4. Therefore, there are no reviews with a rating of 10, and I cannot provide URLs for such reviews.'

In [19]:
bm25_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the John Wick series, the story revolves around a skilled assassin named John Wick. The series depicts his actions following the death of his wife and the subsequent events that lead him into a series of intense and often brutal confrontations with other assassins and criminal elements. The movies are known for their highly choreographed action scenes, emotional depth, and a detailed underworld of organized crime and assassin societies. Over the course of the series, John Wick battles various enemies, seeks revenge, and navigates a complex world of assassin rules and alliances.'

It's not clear that this is better or worse - but the `I don't know` isn't great!

## Task 6: Contextual Compression (Using Reranking)

Contextual Compression is a fairly straightforward idea: We want to "compress" our retrieved context into just the most useful bits.

There are a few ways we can achieve this - but we're going to look at a specific example called reranking.

The basic idea here is this:

- We retrieve lots of documents that are very likely related to our query vector
- We "compress" those documents into a smaller set of *more* related documents using a reranking algorithm.

We'll be leveraging Cohere's Rerank model for our reranker today!

All we need to do is the following:

- Create a basic retriever
- Create a compressor (reranker, in this case)

That's it!

Let's see it in the code below!

In [20]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=naive_retriever
)

Let's create our chain again, and see how this does!

In [21]:
contextual_compression_retrieval_chain = (
    {"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [22]:
contextual_compression_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, people generally liked John Wick. The first two reviews are highly positive, with ratings of 9 and 10 out of 10, praising its action sequences, style, and overall entertainment value. However, there is a less favorable review of the third film, with a rating of 5 out of 10, mentioning that "the magic is gone." Overall, the majority of the reviews suggest that people generally enjoyed the film series when it was first released.'

In [23]:
contextual_compression_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there are reviews with a rating of 10. Here are the URLs to those reviews:\n\n1. Review Title: "A Masterpiece & Brilliant Sequel"  \n   URL: [https://yourdomain.com/review/rw4854296/?ref_=tt_urv](https://yourdomain.com/review/rw4854296/?ref_=tt_urv)\n\n2. Review Title: "Most American action flicks released these days have poor screenplays and overuse computer-generated imagery."  \n   URL: [https://yourdomain.com/review/rw4860412/?ref_=tt_urv](https://yourdomain.com/review/rw4860412/?ref_=tt_urv)\n\nPlease note that the URLs are based on the review data provided.'

In [24]:
contextual_compression_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the "John Wick" film series, the story follows a retired hitman named John Wick (played by Keanu Reeves) who is drawn back into a violent world of crime. In the first movie, he seeks revenge after gangsters kill his beloved dog and steal his car, which are the last remnants of his late wife. As he pursues justice, he becomes entangled with the Russian mafia and a criminal underworld governed by strict rules and codes. He eventually seeks vengeance against those who wronged him, leading to a series of brutal action scenes and confrontations. In the second installment, Wick is drawn back into the criminal underworld when Santino D\'Antonio, a mobster, seeks his help to eliminate his sister so he can rise to power. When Wick refuses and later executes Santino\'s request, Santino retaliates by placing a contract on Wick\'s life, putting him in the crosshairs of professional killers. Throughout the series, Wick\'s actions have significant consequences, and he navigates a dangerous under

We'll need to rely on something like Ragas to help us get a better sense of how this is performing overall - but it "feels" better!

## Task 7: Multi-Query Retriever

Typically in RAG we have a single query - the one provided by the user.

What if we had....more than one query!

In essence, a Multi-Query Retriever works by:

1. Taking the original user query and creating `n` number of new user queries using an LLM.
2. Retrieving documents for each query.
3. Using all unique retrieved documents as context

So, how is it to set-up? Not bad! Let's see it down below!



In [25]:
from langchain.retrievers.multi_query import MultiQueryRetriever

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=naive_retriever, llm=chat_model
)

In [26]:
multi_query_retrieval_chain = (
    {"context": itemgetter("question") | multi_query_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [27]:
multi_query_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked John Wick. Many reviews mention that the film is highly entertaining, well-choreographed, stylish, and fun, especially highlighting its action sequences and Keanu Reeves' performance. Several reviews give high ratings and express enthusiasm for the movie, indicating that overall, viewers enjoyed it. However, there are some mixed opinions and critiques about the later sequels, but the general sentiment toward the first film and the series as a whole suggests that people generally liked John Wick."

In [28]:
multi_query_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there are reviews with a rating of 10. One such review is for "John Wick 3" with the URL: /review/rw4854296/?ref_=tt_urv.'

In [29]:
multi_query_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In the John Wick series, the story revolves around John Wick, a retired assassin who is drawn back into a world of violence and crime after personal tragedies and conflicts with powerful criminal organizations. The movies depict his quest for vengeance, his battles against numerous enemies, and the complex rules and hierarchies of the assassination underworld. The series is known for its stylized, high-octane action sequences, choreographed combat scenes, and a detailed depiction of a secret society of killers. Each installment explores the consequences of Wick's actions and the expanding global universe of assassins, with themes of revenge, loyalty, and the costs of a life of crime."

## Task 8: Parent Document Retriever

A "small-to-big" strategy - the Parent Document Retriever works based on a simple strategy:

1. Each un-split "document" will be designated as a "parent document" (You could use larger chunks of document as well, but our data format allows us to consider the overall document as the parent chunk)
2. Store those "parent documents" in a memory store (not a VectorStore)
3. We will chunk each of those documents into smaller documents, and associate them with their respective parents, and store those in a VectorStore. We'll call those "child chunks".
4. When we query our Retriever, we will do a similarity search comparing our query vector to the "child chunks".
5. Instead of returning the "child chunks", we'll return their associated "parent chunks".

Okay, maybe that was a few steps - but the basic idea is this:

- Search for small documents
- Return big documents

The intuition is that we're likely to find the most relevant information by limiting the amount of semantic information that is encoded in each embedding vector - but we're likely to miss relevant surrounding context if we only use that information.

Let's start by creating our "parent documents" and defining a `RecursiveCharacterTextSplitter`.

In [30]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models

parent_docs = documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

We'll need to set up a new QDrant vectorstore - and we'll use another useful pattern to do so!

> NOTE: We are manually defining our embedding dimension, you'll need to change this if you're using a different embedding model.

In [31]:
client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="full_documents",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

parent_document_vectorstore = Qdrant(
    collection_name="full_documents", embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), client=client
)

/var/folders/54/x_4x8xps7lqf2rzdt9mhf_y80000gn/T/ipykernel_45566/3574430551.py:8: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  parent_document_vectorstore = Qdrant(


Now we can create our `InMemoryStore` that will hold our "parent documents" - and build our retriever!

In [32]:
store = InMemoryStore()

parent_document_retriever = ParentDocumentRetriever(
    vectorstore = parent_document_vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

By default, this is empty as we haven't added any documents - let's add some now!

In [33]:
parent_document_retriever.add_documents(parent_docs, ids=None)

We'll create the same chain we did before - but substitute our new `parent_document_retriever`.

In [34]:
parent_document_retrieval_chain = (
    {"context": itemgetter("question") | parent_document_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's give it a whirl!

In [35]:
parent_document_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people's opinions on John Wick are mixed. Some reviews are very positive, praising the series and the first film highly, while at least one review is quite negative, criticizing John Wick 4 specifically. Overall, it seems that many people do like or appreciate the series, but there are also some who did not enjoy John Wick 4."

In [36]:
parent_document_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. The URL to that review is /review/rw4854296/?ref_=tt_urv.'

In [37]:
parent_document_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the John Wick movies, the story follows John Wick, a retired assassin played by Keanu Reeves. In the first film, he comes out of retirement after a gang of Russian mobsters steals his car and kills his dog, which was a gift from his deceased wife, leading him to seek violent revenge. The film is known for its intense action, shootouts, and fighting scenes.\n\nIn the second film, the story continues with John Wick still involved in the world of assassins. He is compelled to assist a former associate in Italy to settle an old debt, which involves traveling to Italy, Canada, and Manhattan, and fighting numerous enemies. The movie emphasizes the consequences of his actions and features even more action and violence.\n\nThe third film, not described in detail here but part of the series, continues to follow John Wick as he faces new enemies and challenges while trying to escape the deadly world he’s entangled in.\n\nOverall, the series depicts John Wick’s relentless quest for vengeance,

Overall, the performance *seems* largely the same. We can leverage a tool like [Ragas]() to more effectively answer the question about the performance.

## Task 9: Ensemble Retriever

In brief, an Ensemble Retriever simply takes 2, or more, retrievers and combines their retrieved documents based on a rank-fusion algorithm.

In this case - we're using the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.

Setting it up is as easy as providing a list of our desired retrievers - and the weights for each retriever.

In [38]:
from langchain.retrievers import EnsembleRetriever

retriever_list = [bm25_retriever, naive_retriever, parent_document_retriever, compression_retriever, multi_query_retriever]
equal_weighting = [1/len(retriever_list)] * len(retriever_list)

ensemble_retriever = EnsembleRetriever(
    retrievers=retriever_list, weights=equal_weighting
)

We'll pack *all* of these retrievers together in an ensemble.

In [39]:
ensemble_retrieval_chain = (
    {"context": itemgetter("question") | ensemble_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

Let's look at our results!

In [40]:
ensemble_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

"Based on the reviews provided, people generally liked John Wick. Many reviews are highly positive, praising its action sequences, style, and Keanu Reeves' performance, with ratings often around 8 or 9 out of 10. However, there are some negative reviews as well, with a few people giving low scores or criticizing the plot and action scenes. Overall, the majority of the reviews indicate that people appreciated and enjoyed the film series."

In [41]:
ensemble_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there are reviews with a rating of 10. One such review is for "John Wick 3," and the URL to that review is:\n\n[https://example.com/review/rw4854296/?ref_=tt_urv](https://example.com/review/rw4854296/?ref_=tt_urv)\n\n(Note: The actual URL based on the data provided is "/review/rw4854296/?ref_=tt_urv". If you need the full web address, it would depend on the website\'s base URL.)'

In [42]:
ensemble_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

'In the John Wick film series, the story centers around John Wick, a retired and highly skilled assassin. The key events are as follows:\n\n- In the first film, John Wick comes out of retirement after thugs steal his car and kill his dog, which was a gift from his deceased wife. This act of violence propels him into a violent quest for revenge, leading to a series of brutal confrontations with gangsters and hitmen.\n\n- The second film continues shortly after the first, with John being pressured to help a crime boss in Italy by killing his target to settle an old debt. This leads to further violence, including a massive amount of combat with many assassins hunting him due to a contract on his life.\n\n- In the third film, John seeks to escape his violent life but is drawn back into the underworld when a former ally forces him to kill his sister in exchange for a favor. After completing the task, he becomes a target of a global bounty, and his fight for survival continues with intense a

## Task 10: Semantic Chunking

While this is not a retrieval method - it *is* an effective way of increasing retrieval performance on corpora that have clean semantic breaks in them.

Essentially, Semantic Chunking is implemented by:

1. Embedding all sentences in the corpus.
2. Combining or splitting sequences of sentences based on their semantic similarity based on a number of [possible thresholding methods](https://python.langchain.com/docs/how_to/semantic-chunker/):
  - `percentile`
  - `standard_deviation`
  - `interquartile`
  - `gradient`
3. Each sequence of related sentences is kept as a document!

Let's see how to implement this!

> NOTE: You do not need to run this cell if you're running this locally

In [43]:
#!pip install -qU langchain_experimental

We'll use the `percentile` thresholding method for this example which will:

Calculate all distances between sentences, and then break apart sequences of setences that exceed a given percentile among all distances.

In [43]:
from langchain_experimental.text_splitter import SemanticChunker

semantic_chunker = SemanticChunker(
    embeddings,
    breakpoint_threshold_type="percentile"
)

Now we can split our documents.

In [44]:
semantic_documents = semantic_chunker.split_documents(documents)

Let's create a new vector store.

In [45]:
semantic_vectorstore = Qdrant.from_documents(
    semantic_documents,
    embeddings,
    location=":memory:",
    collection_name="JohnWickSemantic"
)

We'll use naive retrieval for this example.

In [46]:
semantic_retriever = semantic_vectorstore.as_retriever(search_kwargs={"k" : 10})

In [47]:
retrieve_test = semantic_retriever.get_relevant_documents("What happened in John Wick?")
print(retrieve_test)

/var/folders/54/x_4x8xps7lqf2rzdt9mhf_y80000gn/T/ipykernel_45566/489839785.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieve_test = semantic_retriever.get_relevant_documents("What happened in John Wick?")


[Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': '2025-05-13T10:11:07.491393', '_id': '9394ff4cb8cc49f9816159bb25d3cf93', '_collection_name': 'JohnWickSemantic'}, page_content="John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity."), Document(metadata={'source': 'john_wick_2.csv', 'row': 19, 'Review_Date': '29 November 2020', 'Review_Title': ' John Wick Kills A Lot Of People\n', 'Review_Url': '/review/rw6

Finally we can create our classic chain!

In [48]:
semantic_retrieval_chain = (
    {"context": itemgetter("question") | semantic_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

And view the results!

In [49]:
semantic_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, people generally liked John Wick. Many reviews are highly positive, praising the action, style, and entertainment value of the films, with ratings often around 8 to 10. However, there are some negative opinions as well, with a few reviews giving low ratings. Overall, the majority of reviews indicate that people tend to enjoy John Wick.'

In [50]:
semantic_retrieval_chain.invoke({"question" : "Do any reviews have a rating of 10? If so - can I have the URLs to those reviews?"})["response"].content

'Yes, there is a review with a rating of 10. The URL to that review is: /review/rw4854296/?ref_=tt_urv'

In [51]:
semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})["response"].content

"In the movie John Wick, the story centers around a retired assassin named John Wick, played by Keanu Reeves. The film begins with a young Russian-American punk stealing Wick's car and later, along with some goons, breaking into his home, beating him up, killing his dog, and stealing his car. The dog was his last meaningful connection to his past life, making its death a catalyst for his quest for revenge.\n\nJohn Wick, who is highly skilled and legendary in his field, then comes out of retirement to seek vengeance against those who wronged him. As the story unfolds, he wages a relentless and violent war against the Russian gangsters involved, demonstrating his lethal capabilities and uncovering a criminal underworld filled with powerful and connected figures. The movie showcases intense action sequences, stylish violence, and a noir-inspired world as Wick's pursuit of justice puts everyone in his path at risk, ultimately revealing the depth of his skills and the consequences of his pa

In [52]:
practice_line = semantic_retrieval_chain.invoke({"question" : "What happened in John Wick?"})

practice_line_context = practice_line["context"]
practice_line_context_clean = [context.page_content for context in practice_line_context]

practice_line_context_clean


["John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.",
 ": 19\nReview: If you've seen the first John Wick movie, you know that Keanu Reeves is John Wick, a retired assassin who comes out of retirement when someone kills his dog. In this one, which begins a week later, matters are still reverberating, and some one has stolen his car, which calls for a lot of carnage.",
 ': 5\nReview: Ultra-violent first entry with lots of killings, thrills , noisy action , suspense , and crossfire . In this original John Wick (2014) , an ex-hit-man comes out of retirement to track down the gangsters that killed his dog and took everything from him . With the u

In [53]:
len(practice_line_context_clean)

10

In [54]:
len(practice_line_context)

10

In [55]:
practice_line_context_clean[0]

"John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity."

# 🤝 Breakout Room Part #2

#### 🏗️ Activity #1

Your task is to evaluate the various Retriever methods against eachother.

You are expected to:

1. Create a "golden dataset"
 - Use Synthetic Data Generation (powered by Ragas, or otherwise) to create this dataset
2. Evaluate each retriever with *retriever specific* Ragas metrics
 - Semantic Chunking is not considered a retriever method and will not be required for marks, but you may find it useful to do a "semantic chunking on" vs. "semantic chunking off" comparision between them
3. Compile these in a list and write a small paragraph about which is best for this particular data and why.

Your analysis should factor in:
  - Cost
  - Latency
  - Performance

> NOTE: This is **NOT** required to be completed in class. Please spend time in your breakout rooms creating a plan before moving on to writing code.

##### HINTS:

- LangSmith provides detailed information about latency and cost.

### Let's first create a golden dataset using Ragas knowledge graph


In [56]:
# Using gpt 4.1-nano to prevent rate limit errors

from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# Configure the model for better JSON responses to prevent errors
generator_llm = LangchainLLMWrapper(
    ChatOpenAI(
        model="gpt-4.1-nano",
        temperature=0,  
        model_kwargs={"response_format": {"type": "json_object"}}  # Force JSON format
    )
)
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [57]:
from ragas.testset.synthesizers import SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]


In [60]:
from ragas.testset import TestsetGenerator


generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(documents, testset_size=10, query_distribution=query_distribution)

Applying SummaryExtractor:   0%|          | 0/44 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/100 [00:00<?, ?it/s]

Node f1f8bec7-4bf1-4b34-a782-d05774d5af29 does not have a summary. Skipping filtering.
Node 9e526faf-b31f-4a94-9d06-90f7387bef0e does not have a summary. Skipping filtering.
Node 58f636a0-f58c-42c2-8820-78cae6140e5a does not have a summary. Skipping filtering.
Node b73a7947-8509-4b26-b336-cc08b2861919 does not have a summary. Skipping filtering.
Node 5de52888-2255-43bc-98e1-776417795f44 does not have a summary. Skipping filtering.
Node 0c22e3fd-aa69-417e-832b-96b9c1287a11 does not have a summary. Skipping filtering.
Node f1738415-3604-45d1-8ef3-ba8c536342c7 does not have a summary. Skipping filtering.
Node 77629322-3914-4e4f-9962-e19e53eb7f52 does not have a summary. Skipping filtering.
Node 68052181-b8d4-4688-a405-74a381452b0b does not have a summary. Skipping filtering.
Node ff0e36b0-d9ce-4452-91e5-6898bf472cec does not have a summary. Skipping filtering.
Node 71d70240-a587-49b0-a2cd-c76c5c4aa100 does not have a summary. Skipping filtering.
Node a372335a-00cd-4f47-b526-e8a8df1b3976 d

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/217 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [61]:
dataset.to_pandas()


,user_input,reference_contexts,reference,synthesizer_name
0,Can you explain why the movie John Wick is con...,[: 0\nReview: The best way I can describe John...,John Wick (Reeves) is out to seek revenge on t...,single_hop_specifc_query_synthesizer
1,Whaat is the genral idea behnd John Wick and w...,[: 2\nReview: With the fourth installment scor...,"The context indicates that ""John Wick"" is a fi...",single_hop_specifc_query_synthesizer
2,Whaat makes John Wick so special despite its s...,[: 3\nReview: John wick has a very simple reve...,"John Wick has a very simple revenge story, sum...",single_hop_specifc_query_synthesizer
3,In the context of the film series featuring Jo...,[: 4\nReview: Though he no longer has a taste ...,The review mentions that John Wick and his dog...,single_hop_specifc_query_synthesizer
4,How does the character Wick evolve throughout ...,[: 5\nReview: Ultra-violent first entry with l...,The context describes John Wick as an ex-hit-m...,single_hop_specifc_query_synthesizer
5,How does Director Chad Stahelski's work in Joh...,[<1-hop>\n\n: 19\nReview: John Wick: Chapter 4...,Director Chad Stahelski's work in John Wick: C...,multi_hop_abstract_query_synthesizer
6,Considering the consistent critical reception ...,[<1-hop>\n\n: 20\nReview: In a world where mov...,The review highlights that the 'John Wick' ser...,multi_hop_abstract_query_synthesizer
7,Considering the consistent series quality of t...,[<1-hop>\n\n: 20\nReview: In a world where mov...,The John Wick series has maintained remarkable...,multi_hop_abstract_query_synthesizer
8,Hwo does Chapter 3 - Parabellum show the conse...,[<1-hop>\n\n: 24\nReview: John Wick: Chapter 3...,Chapter 3 - Parabellum illustrates the consequ...,multi_hop_specific_query_synthesizer
9,How does the portrayal of Russian gangsters in...,[<1-hop>\n\n: 10\nReview: Wow what a great sur...,The context describes John Wick's conflict wit...,multi_hop_specific_query_synthesizer


In [62]:
dataset.to_pandas()['reference_contexts'].iloc[0]

[": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity."]

### Important Note:

Notice that in all the chains above, we are sending the LLM not the clean version of the context (i.e. the '.page_content' version). Respectfully, perhaps this was a typo since this isn't what we've done previously (also for a previous cohort for this session, they did it with the clean '.page_content' version). 

So the question becomes, when doing our evaluations, do we give Ragas the clean '.page_content' version of the context? I think so because that was probably the intent. So that's what I'll do below (please don't mark off point for this, I know how to do it either way (the other option is to just convert the Document objects to str))

In [66]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

In [67]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"Session 13 AIE6 - {uuid4().hex[0:8]}"

In [68]:
from ragas import EvaluationDataset

# We use the same model that we used to generate our synthetic data to be our judge

from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano", max_tokens=4096))


In [69]:
# We evaluate the baseline using our key 6 Ragas RAG metrics

from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextEntityRecall, NoiseSensitivity
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=600)

In [70]:
import copy
import pandas as pd


# Define retrieval chains to evaluate
retrieval_chains = [
    ("naive_retrieval_chain", naive_retrieval_chain),
    ("bm25_retrieval_chain", bm25_retrieval_chain),
    ("contextual_compression_retrieval_chain", contextual_compression_retrieval_chain),
    ("multi_query_retrieval_chain", multi_query_retrieval_chain),
    ("parent_document_retrieval_chain", parent_document_retrieval_chain),
    ("ensemble_retrieval_chain", ensemble_retrieval_chain),
    ("semantic_retrieval_chain", semantic_retrieval_chain)
]

results = []

# Evaluate each retrieval chain
for chain_name, chain in retrieval_chains:
    # Create dataset copy so we don't have mutation issues
    dataset_copy = copy.deepcopy(dataset)
    
    # Process dataset with current chain
    for test_row in dataset_copy:
        response = chain.invoke({"question": test_row.eval_sample.user_input})
        test_row.eval_sample.retrieved_contexts = [ctx.page_content for ctx in response["context"]]
        test_row.eval_sample.response = response["response"].content
    
    # Run evaluation
    eval_dataset = EvaluationDataset.from_pandas(dataset_copy.to_pandas())
    result = evaluate(
        dataset=eval_dataset,
        metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextEntityRecall(), NoiseSensitivity()],
        llm=evaluator_llm,
        run_config=custom_run_config
    )
    
    # Store results with chain identifier
    results.append({
    "Chain": chain_name,
    "context_recall": result["context_recall"],
    "faithfulness": result["faithfulness"],
    "factual_correctness(mode=f1)": result["factual_correctness(mode=f1)"],
    "answer_relevancy": result["answer_relevancy"],
    "context_entity_recall": result["context_entity_recall"],
    "noise_sensitivity(mode=relevant)": result["noise_sensitivity(mode=relevant)"]})

# Create comparison table
df = pd.DataFrame(results).set_index("Chain")
df = df[[
    'context_recall',
    'faithfulness', 
    'factual_correctness(mode=f1)',
    'answer_relevancy',
    'context_entity_recall',
    'noise_sensitivity(mode=relevant)'
]].round(4)

df


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[22]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[34]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[40]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[52]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[58]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[35]: ValueError(setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (10,) + inhomogeneous part.)
Exception raised in Job[11]: ValueError(setting an array element with a sequence. The requested array ha

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[64]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[34]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[40]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[58]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[64]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[53]: ValueError(setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (13,) + inhomogeneous part.)


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Exception raised in Job[40]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[58]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)


,context_recall,faithfulness,factual_correctness(mode=f1),answer_relevancy,context_entity_recall,noise_sensitivity(mode=relevant)
Chain,,,,,,
naive_retrieval_chain,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9375, 1.0, 1.0, 1.0, 1.0, 0.954545454545454...","[0.58, 0.84, 0.44, 0.75, 0.88, 0.72, 0.85, 0.6...","[0.9445701416835623, 0.8966078832256427, 0.940...","[0.999999995, 0.14285714265306124, 0.666666664...","[0.3125, nan, 0.26666666666666666, 0.357142857..."
bm25_retrieval_chain,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.6666666666666...","[1.0, 1.0, 0.9, 0.0, 1.0, 0.9333333333333333, ...","[0.82, 0.76, 0.54, 0.92, 0.67, 0.89, 0.32, 0.4...","[0.961892086096711, 0.8957964187828619, 0.9601...","[0.4999999975, 0.14285714265306124, 0.66666666...","[0.0, 0.25, 0.5, 0.0, 0.16666666666666666, 0.6..."
contextual_compression_retrieval_chain,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.8333333333333334, 0.9333333333333333, ...","[0.89, 0.81, 0.67, 0.81, 0.58, 0.42, 0.5, 0.63...","[0.9590799021284276, 0.950450543482812, 0.9693...","[0.4999999975, 0.0, 0.9999999966666667, 0.7499...","[0.13333333333333333, 0.3333333333333333, 0.33..."
multi_query_retrieval_chain,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.75,...","[0.6, 0.73, 0.94, 0.77, 0.95, 0.59, 0.81, 0.62...","[0.961892086096711, 0.9022383848052451, 0.9598...","[0.999999995, 0.14285714265306124, 0.666666664...","[0.625, 0.8888888888888888, 0.5, 0.38461538461..."
parent_document_retrieval_chain,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.9166666666666666, 1.0, 1.0, 0.5454545454545...","[0.58, 0.61, 0.86, 0.83, 0.5, 0.77, 0.53, 0.4,...","[0.961892086096711, 0.9048378074783896, 0.9693...","[0.4999999975, 0.14285714265306124, 0.66666666...","[0.5833333333333334, 0.7777777777777778, 0.111..."
ensemble_retrieval_chain,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 0.9375, 1.0, 1.0, 0....","[0.79, 0.18, 0.57, 0.86, 0.64, 0.93, 0.69, 0.7...","[0.9618791268483196, 0.8957925230671618, 0.969...","[0.4999999975, 0.14285714265306124, 0.66666666...","[0.36363636363636365, 0.875, 0.571428571428571..."
semantic_retrieval_chain,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[1.0, 0.9333333333333333, 1.0, 1.0, 1.0, 1.0, ...","[0.86, 0.45, 0.81, 0.69, 0.86, 0.55, 0.82, 0.3...","[0.9637899969748339, 0.928933927503548, 0.9598...","[0.999999995, 0.12499999984374999, 0.333333332...","[0.42857142857142855, 0.5, 0.14285714285714285..."


Oops, I forgot about taking the average of the lists! Let's fix this!

In [83]:
# Create comparison table
df_test = pd.DataFrame(results).set_index("Chain")
df_test = df_test[[
    'context_recall',
    'faithfulness', 
    'factual_correctness(mode=f1)',
    'answer_relevancy',
    'context_entity_recall',
    'noise_sensitivity(mode=relevant)'
]].round(4)

In [84]:
import pandas as pd
import numpy as np

def process_cell(cell):
    """Process list cells to return mean, handling NaNs and np.float64"""
    if isinstance(cell, list):
        cleaned = []
        for val in cell:
            # Handle numpy types and check for NaN
            if isinstance(val, np.float64):
                val = float(val)
            if pd.notna(val):  # Check for both numpy and pandas NaNs
                cleaned.append(val)
        return np.mean(cleaned) if cleaned else np.nan
    return cell  # Return non-list values as-is

# Define metric columns to process
metric_columns = [
    'context_recall',
    'faithfulness', 
    'factual_correctness(mode=f1)',
    'answer_relevancy',
    'context_entity_recall',
    'noise_sensitivity(mode=relevant)'
]

# Create clean dataframe with averages
df_clean = df_test.copy()
for col in metric_columns:
    df_clean[col] = df_clean[col].apply(process_cell)

# Formatting
df_clean = df_clean.round(4)
df_clean = df_clean[metric_columns]  # Ensure column order
df_clean.index = df.index  # Preserve original index

# Display settings for clean output
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', 20)

df_clean


,context_recall,faithfulness,factual_correctness(mode=f1),answer_relevancy,context_entity_recall,noise_sensitivity(mode=relevant)
Chain,,,,,,
naive_retrieval_chain,1.0000,0.9584,0.7427,0.9368,0.3789,0.1798
bm25_retrieval_chain,0.9697,0.8636,0.6864,0.9388,0.2802,0.1480
contextual_compression_retrieval_chain,1.0000,0.8938,0.7000,0.9391,0.3474,0.2420
multi_query_retrieval_chain,1.0000,0.9773,0.7745,0.9356,0.4371,0.3790
parent_document_retrieval_chain,1.0000,0.8033,0.6609,0.8536,0.3253,0.1980
ensemble_retrieval_chain,1.0000,0.9659,0.6900,0.9383,0.2807,0.3494
semantic_retrieval_chain,1.0000,0.9939,0.6945,0.9419,0.3435,0.2374


I noticed that the LangSmith tracing never happened! So I looked at their website and will try to follow how they say and will do some tracing again (but not the evaluation datasets because that took 40 minutes and failed a couple times before actually working!)

In [89]:
import os
import getpass

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith API Key:")

In [90]:
from uuid import uuid4

os.environ["LANGSMITH_PROJECT"] = f"Session 13 AIE6 - {uuid4().hex[0:8]}"

In [91]:
import os
import urllib.parse

project_name = urllib.parse.quote(os.environ["LANGSMITH_PROJECT"])
langsmith_url = f"https://smith.langchain.com/projects/{project_name}/traces"
print(f"You can view your traces at: {langsmith_url}")


You can view your traces at: https://smith.langchain.com/projects/Session%2013%20AIE6%20-%203fb6629a/traces


In [92]:
naive_retrieval_chain.invoke({"question" : "Did people generally like John Wick?"})["response"].content

'Based on the reviews provided, people generally liked John Wick. Many reviews rate it highly, with some giving perfect or near-perfect scores, and praise its style, action sequences, and overall entertainment value. However, there are a few mixed opinions and some lower ratings, but the overall sentiment suggests that the film was well-received and enjoyed by most.'